# Recomindation System Task

## 1 - Packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
nltk.download('stopwords')
from flask import Flask, jsonify, request, redirect
import os

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/abdelrahman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 2 - Read Data

In [2]:
file_name = 'jobs_data.csv'
file_dir = !pwd
file_dir = file_dir[0] + '/' + file_name

In [3]:
df_file = pd.read_csv(file_dir)

In [4]:
print("First 5 entery of our Data\n")
df_file.head()

First 5 entery of our Data



Unnamed: 0                                    title  \
0           0                 Full Stack PHP Developer   
1           1  CISCO Collaboration Specialist Engineer   
2           2            Senior Back End-PHP Developer   
3           3                              UX Designer   
4           4                      Java Technical Lead   

                                         jobFunction  \
0  ['Engineering - Telecom/Technology', 'IT/Softw...   
1  ['Installation/Maintenance/Repair', 'IT/Softwa...   
2  ['Engineering - Telecom/Technology', 'IT/Softw...   
3  ['Creative/Design/Art', 'IT/Software Developme...   
4  ['Engineering - Telecom/Technology', 'IT/Softw...   

                                            industry  
0  ['Computer Software', 'Marketing and Advertisi...  
1                ['Information Technology Services']  
2       ['Computer Software', 'Computer Networking']  
3  ['Computer Software', 'Information Technology ...  
4  ['Computer Software', 'Information Technology ...

In [5]:
df_file.describe()

Unnamed: 0
count  10870.000000
mean    5434.500000
std     3138.043047
min        0.000000
25%     2717.250000
50%     5434.500000
75%     8151.750000
max    10869.000000

In [6]:
print("Total data examples are\n", len(df_file))

Total data examples are
 10870


In [7]:
print("Unique Titles \n", len(df_file['title'].unique()))

Unique Titles 
 3230


In [8]:
print("Unique Titles \n", len(df_file['jobFunction'].unique()))

Unique Titles 
 836


In [9]:
df_file = df_file.drop_duplicates(subset='title')
print("Total data examples after remove dublicated are\n", len(df_file))

Total data examples after remove dublicated are
 3230


## Drop columns

**some of columns are deleted that has no meaning and imact on our data like Unnamed: 0 and industry**

# PreProcessing Data

## before we start to take a step  we need to handle our data

**some of these handles are:**
- braces
- punctuation
- quotes
- Tokenization — convert sentences to words
- Stemming  back word to its root like studying - study
-  under_score ,stop words and other depends on user input because our data cleaned some of these 


In [10]:
def clean_text(words):
    '''
    a fcuntion return cleaned text from punctuation, braces, quotes and others
    argument:
        string of words
    return:
        cleaned string
    '''
    words = list(words)
    reqular_expression = '][,"\'/-_'
    cleaned_words = ""
    for word in words:
        one_cleaned_word = ""
        word = word.replace('/', ' ')
        word = word.replace(']', ' ')
        word = word.replace('[', ' ')
        for c in word:
            if not c.isdigit() and c not in reqular_expression:
                one_cleaned_word +=c
        cleaned_words +=one_cleaned_word
    return cleaned_words

In [11]:
def remove_stop_words(words):
    '''
    a function return list of words without stop words
    stop words like the,and, over and others english stopwords
    argument:
        string of words
    return:
        list of words
    '''
    words = word_tokenize(words)
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    return words

## clean and drop dublicated rows from our data
**some of our rows are repeated with the same data and we do not nead these dublicated**

In [12]:

def file_cleaning(file):
    '''
    a function that return over all cleaned file
    argument:
        csv file
    return:
        cleaned file
    '''
    for key, value in file.iterrows():
        value['title'] = clean_text(value['title'])
        value['jobFunction'] = clean_text(value['jobFunction'])
        cleaned_title = remove_stop_words(value['title'])
        cleaned_function = remove_stop_words(value['jobFunction'])
        value['title'] = ' '.join(map(str,cleaned_title)).lower()
        value['jobFunction'] = ' '.join(map(str,cleaned_function)).lower()
    return file

In [13]:
def compute_jaccard_similarity_score(x, y):
    """
        a function to return similarities between two string
    argument:
        x as  string which user_job_title
        y for each of job title in our file
    """
    intersection_cardinality = len(set(x).intersection(set(y)))
    union_cardinality = len(set(x).union(set(y)))
    return intersection_cardinality / float(union_cardinality)


In [14]:
def weighted_similarity(job_title, title_list):
    '''
    function take a string and comapre its similarity
    similarity for each job title with this title
    argument:
        user job_title, and all of our job_title
    return:
        sordted list of set with the weights and the index meet this weight in our data 
    '''
    weighted_similarity = []
    for indx, title in enumerate(title_list):
        weights = compute_jaccard_similarity_score(title, job_title)
        weighted_similarity.append((weights, indx))
        weighted_similarity.sort(reverse=True)
    return weighted_similarity

In [15]:
def most_related_jobFunction(job_title, df_file_updated):
    '''
        this function return most frequent job function to user
        argument:
            user job_title, our data file
        return:
            most related 5 job function
    '''
    all_related_weighted_similarity = weighted_similarity(job_title, df_file_updated['title'])
    most_related_jobs = []
    for job in range(0, 2):
        most_related_jobs.append(df_file_updated['jobFunction'][all_related_weighted_similarity[job][1]])
    return most_related_jobs


In [16]:
def recommendation_model(file_to_clean, user_job_title):
    '''
        1 - clean the file data
        2 - read cleaned file
        3-  remove null values from cleaned file
        4 - get related job function
        argument:
            file we need to clean for our recommendation
        return:
            related job function
        
    '''
    df_file = file_to_clean.drop(['Unnamed: 0', 'industry'],axis=1)
    df_file = file_cleaning(df_file)

    dir_file = !pwd
    path = dir_file[0]
    df_file.to_csv(path + '/jobs_data_updated.csv', index=False)
    df_file_updated = pd.read_csv(path+ '/jobs_data_updated.csv')
    
    df_file_updated =  df_file_updated.dropna()
    
    user_job_titl = user_job_title
    most_jobfunction = most_related_jobFunction(user_job_title, df_file_updated)

    return most_jobfunction

In [17]:
most_jobfunction = recommendation_model(df_file, "CISCO Collaboration Specialist Engineer")
job_functions= set()
for val in most_jobfunction:
    words = remove_stop_words(val)
    for word in words:
        job_functions.add(word.capitalize())
job_functions = list(job_functions)
print("\n\nMost relevant jobs are\n")
print(job_functions)



Most relevant jobs are

['Mechanical', 'Telecom', 'Technology', 'Repair', 'Installation', 'Maintenance', 'Software', 'Engineering', 'Development', 'Electrical']


In [18]:
most_jobfunction = recommendation_model(df_file, "Accountant")
job_functions= set()
for val in most_jobfunction:
    words = remove_stop_words(val)
    for word in words:
        job_functions.add(word.capitalize())
job_functions = list(job_functions)
print("\n\nMost relevant jobs are\n")
print(job_functions)



Most relevant jobs are

['Chain', 'Civil', 'Finance', 'Construction', 'Supply', 'Accounting', 'Sales', 'Architecture', 'Logistics', 'Engineering', 'Retail']
